<a href="https://colab.research.google.com/github/ashkanb77/cats-vs-dogs/blob/main/CatVSDog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d shaunthesheep/microsoft-catsvsdogs-dataset

mkdir: cannot create directory ‘/root/.kaggle’: File exists
 98% 769M/788M [00:06<00:00, 132MB/s]
100% 788M/788M [00:06<00:00, 126MB/s]


In [3]:
def prepare_dataset(test_size):
  import zipfile
  import os
  import shutil
  from random import randint

  zipfile.ZipFile('microsoft-catsvsdogs-dataset.zip').extractall('cats_vs_dogs')

  os.makedirs('cats_vs_dogs/train')

  shutil.move('cats_vs_dogs/PetImages/Cat', 'cats_vs_dogs/train')
  shutil.move('cats_vs_dogs/PetImages/Dog', 'cats_vs_dogs/train')

  os.makedirs('cats_vs_dogs/test/Cat')
  os.makedirs('cats_vs_dogs/test/Dog')

  l = os.listdir('cats_vs_dogs/train/Cat')
  n = int(test_size * len(l))
  for i in range(n):
    r = l.pop(randint(0, len(l) - 1))
    shutil.move('cats_vs_dogs/train/Cat/' + r, 'cats_vs_dogs/test/Cat/' + r)
  

  l = os.listdir('cats_vs_dogs/train/Dog')
  n = int(test_size * len(l))
  for i in range(n):
    r = l.pop(randint(0, len(l) - 1))
    shutil.move('cats_vs_dogs/train/Dog/' + r, 'cats_vs_dogs/test/Dog/' + r)

prepare_dataset(0.1)

In [4]:
import PIL
import os
from PIL import Image
folder_path = 'cats_vs_dogs/train'
extensions = []
for fldr in os.listdir(folder_path):
    sub_folder_path = os.path.join(folder_path, fldr)
    for filee in os.listdir(sub_folder_path):
        file_path = os.path.join(sub_folder_path, filee)
        print('** Path: {}  **'.format(file_path), end="\r", flush=True)
        try:
          im = Image.open(file_path)
          rgb_im = im.convert('RGB')
        except PIL.UnidentifiedImageError:
          os.remove(file_path)
        if filee.split('.')[1] not in extensions:
            extensions.append(filee.split('.')[1])
    

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [7]:
data_generator = ImageDataGenerator(rescale=1./255.)

In [9]:
train = data_generator.flow_from_directory('cats_vs_dogs/train', class_mode='binary', batch_size=100, target_size=(128, 128))

Found 22499 images belonging to 2 classes.


In [10]:
test = data_generator.flow_from_directory('cats_vs_dogs/test', class_mode='binary', batch_size=100, target_size=(128, 128))

Found 2499 images belonging to 2 classes.


In [11]:
es = EarlyStopping(monitor='val_loss', mode='min', patience=7)
rlr = ReduceLROnPlateau(monitor='val_loss', mode='min', patience=10, min_lr=0.0001, factor=0.2)
check = ModelCheckpoint('cat_vs_dog.h5', monitor='val_loss', save_best_only=True)

In [41]:
model = Sequential([
                    Conv2D(32, (9, 9), activation='relu', input_shape=(128, 128, 3)),
                    Conv2D(64, (5, 5), activation='relu'),
                    MaxPool2D(),
                    Conv2D(64, (3, 3), activation='relu'),
                    MaxPool2D(),
                    Conv2D(128, (3, 3), activation='relu'),
                    MaxPool2D(),
                    Conv2D(256, (3, 3), activation='relu'),
                    MaxPool2D(),
                    Flatten(),
                    Dense(256, activation='tanh'),
                    Dense(128, activation='relu'),
                    Dense(1, activation='sigmoid')
])

In [42]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_52 (Conv2D)          (None, 120, 120, 32)      7808      
                                                                 
 conv2d_53 (Conv2D)          (None, 116, 116, 64)      51264     
                                                                 
 max_pooling2d_49 (MaxPoolin  (None, 58, 58, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_54 (Conv2D)          (None, 56, 56, 64)        36928     
                                                                 
 max_pooling2d_50 (MaxPoolin  (None, 28, 28, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_55 (Conv2D)          (None, 26, 26, 128)     

In [43]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [44]:
history = model.fit(train, steps_per_epoch=len(train), validation_data=test, validation_steps=len(test), epochs=10, callbacks=[es, rlr, check])

Epoch 1/10
225/225 [==============================] - ETA: 0s - loss: 0.6946 - accuracy: 0.5144

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

225/225 [==============================] - 78s 336ms/step - loss: 0.6946 - accuracy: 0.5144 - val_loss: 0.6813 - val_accuracy: 0.5738 - lr: 0.0010
Epoch 2/10
225/225 [==============================] - 73s 324ms/step - loss: 0.6723 - accuracy: 0.5790 - val_loss: 0.6714 - val_accuracy: 0.6230 - lr: 0.0010
Epoch 3/10
225/225 [==============================] - 73s 324ms/step - loss: 0.6328 - accuracy: 0.6415 - val_loss: 0.6215 - val_accuracy: 0.6547 - lr: 0.0010
Epoch 4/10
225/225 [==============================] - 73s 325ms/step - loss: 0.5796 - accuracy: 0.6919 - val_loss: 0.5701 - val_accuracy: 0.7103 - lr: 0.0010
Epoch 5/10
225/225 [==============================] - 73s 323ms/step - loss: 0.5296 - accuracy: 0.7331 - val_loss: 0.5314 - val_accuracy: 0.7351 - lr: 0.0010
Epoch 6/10
225/225 [==============================] - 73s 322ms/step - loss: 0.4885 - accuracy: 0.7640 - val_loss: 0.5085 - val_accuracy: 0.7491 - lr: 0.0010
Epoch 7/10
225/225 [==============================] - 73s 322ms